**IMPORTS**

In [20]:
import numpy as np

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns


# Descrição:
Wines é um dataset disponibilizado pela biblioteca seaborn com 3 classes de vinhos e 13 features. 

# Objetivo:
O desafio é criar um modelo de Machine Learning capaz de interpretar as características do vinho e indicar a qual classe o vinho pertence.

#Etapas
Estratificação dos dados em treinamento, validação e testes;
Criação de um pipeline: Normalização dados, redução de dimensionalidade (PCA) e seleção de um modelo (KNN);
Seleção de hiperparâmetros;
Avaliação de acurácia, UnderFitting e OverFitting;
Calculo do intervalo de confiança



# Documentação Dataset:

https://www.kaggle.com/c/wine-m/data

**LOADING DATA**

In [21]:
wines = datasets.load_wine()
X = wines.data
y = wines.target

##Quantas caracteristicas existem ?
X.shape

(178, 13)

**DATA PREPARATION**

In [26]:
type(wines)

sklearn.utils.Bunch

Como ha 13 caracteristicas seria interessante reduzir a dimensionalidade, vamos optar pelo modelo de PCA para reduzir a dimensionalidade e como classificador utilizaremos o KNN uma vez que todas as caracteristicas são numéricas

In [3]:
###Split dos dados em treinamento, validacao e teste
### Adicionamos um random_state = 1 para garantir a reprodutibilidade da estratificação dos dados, ou seja, sempre estratificar da mesma forma toda vez que o código 
## for executado

X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y) ### 
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.5, random_state=1) ### 

In [4]:
pipe = Pipeline([
        ('z-score', StandardScaler()),
        ('reduce_dim', PCA(n_components=3)),
        ('classify', KNeighborsClassifier(n_neighbors=1))])

**MODEL TRAINING**

In [5]:

pipe.fit(X_train, y_train)

Pipeline(steps=[('z-score', StandardScaler()),
                ('reduce_dim', PCA(n_components=3)),
                ('classify', KNeighborsClassifier(n_neighbors=1))])

In [6]:
pip install hypopt

**SELEÇÃO DE HIPERPARÂMETROS**

In [7]:
## Procura pelos melhores hyperparametros

from hypopt import GridSearch


## Variar os hiperparametros de 1 a 13 para a reducao de dimensionalidade no PC e de 2 a 5 vizinhos mais proximos do KNN
param_grid = {
    'reduce_dim__n_components': [1, 2, 3, 4,5,6,7,8,9,10,11,12,13],
    'classify__n_neighbors': [2, 3, 4, 5]
}

grid = GridSearch(pipe, param_grid=param_grid)

In [8]:
## Procura os hyperparametros utilizando os dados de treino e validando-os com os dados de validacao
grid.fit(X_train, y_train, X_valid, y_valid)

100%|██████████| 52/52 [00:00<00:00, 105.89it/s]


Pipeline(steps=[('z-score', StandardScaler()),
                ('reduce_dim', PCA(n_components=4)),
                ('classify', KNeighborsClassifier(n_neighbors=3))])

In [9]:
## Melhores Hyperparametros do modelo PCA-KNN
print(grid.best_params)

{'classify__n_neighbors': 3, 'reduce_dim__n_components': 4}


Conclusão - Melhores Hiperparametros: 3 vizinhos mais proximos para KNN e reducao de 4 dim para PCA

**ACURÁCIA DO MODELO**

In [10]:
## Acuracia no conjunto de validacao do melhor modelo com conjunto de hyperparametros dentro do grid
print(grid.best_score)

0.967741935483871


**AVALIAÇÃO UNDERFITTING**

O objeto grid = GridSearch ja avalia a acuracia do modelo com dados nao visto anteriormente do conjunto de dados de validacao. Pelo score do conjunto de validacao pode-se concluir que nao há underfitting uma vez que o erro é muito baixo na base de testes / validacao.

In [11]:
#Melhor classificador selecionado a partir da melhor acuracia sobre os dados de validacao
clf = grid.best_estimator_

In [12]:
### Acuracia no conjunto de testes

from sklearn.metrics import accuracy_score

y_test_pred = clf.predict(X_test)
acc_test = accuracy_score(y_test, y_test_pred)

print(acc_test)


0.9259259259259259


**AVALIAÇÃO OVERFITTING**

Avaliando a alta acuracia apontada pelo modelo ao aplica-lo no conjunto de dados de teste e possivel concluir que nao ha overfitting uma vez que o erro de predicao é muito baixo (acuracia alta).

In [13]:
## Retreinamento com os dados de treinamento e validacao - Geralmente aumenta a acuracia no conjunto de testes
clf.fit(X_train_valid, y_train_valid)

Pipeline(steps=[('z-score', StandardScaler()),
                ('reduce_dim', PCA(n_components=4)),
                ('classify', KNeighborsClassifier(n_neighbors=3))])

In [14]:
## Recalculando acuracia no conjunto de testes e validação
y_test_pred = clf.predict(X_test)
acc_test = accuracy_score(y_test, y_test_pred)

print(acc_test)

0.9259259259259259


In [ ]:
Apos a selecao do modelo e o retreinamento com os dados de treinamento e validacao a acuracia final do modelo é de 92.59% na base de teste.

**VERIFICANDO O INTERVALO DE CONFIANÇA**

In [15]:
##Intervalo de Confianca

ci_test = 1.96 * np.sqrt((acc_test*(1-acc_test)) / y_test.shape[0])

test_lower = acc_test-ci_test
test_upper = acc_test+ci_test

print(test_lower, test_upper)

0.8560736711350823 0.9957781807167696


Interpretação:

em 95% das vezes que o treinamento for refeito com outros conjunto de dados a acuracia vai ficar entre 85.6 e 99.6%